# Multi-Agent Ticket Triage System

This notebook demonstrates how to build a **multi-agent system** using Azure AI Agent Service for automated ticket triage. The system uses multiple specialized agents that work together to analyze support tickets and determine:

- **Priority Level** (High/Medium/Low)
- **Team Assignment** (Frontend/Backend/Infrastructure/Marketing)
- **Effort Estimation** (Small/Medium/Large)

## Architecture Overview

The system consists of:
1. **Three Specialist Agents**: Each focused on one aspect of ticket analysis
2. **One Orchestrator Agent**: Uses the specialist agents as tools to provide comprehensive triage
3. **Connected Agent Tools**: Enable the orchestrator to call specialist agents as needed

## 📦 Import Required Libraries and Setup Environment

This cell imports all the necessary libraries for building our multi-agent system and loads environment variables from the `.env` file. We need:

- **Azure AI Agents SDK**: To create and manage multiple AI agents
- **Azure Identity**: For authentication with Azure services
- **Environment variables**: Project endpoint and model deployment details

In [ ]:
#r "nuget: Azure.AI.Agents.Persistent, 1.2.0-beta.8"
#r "nuget: Azure.Identity"
#r "nuget: dotenv.net"

using Azure;
using Azure.AI.Agents.Persistent;
using Azure.Identity;
using dotenv.net;
using System.IO;

// Load environment variables from .env file
DotEnv.Load(new DotEnvOptions(envFilePaths: new[] { Path.Combine(".","..", ".env") })); 

// Get tenant Id and project endpoint - fix the env variable names as needed
var tenantId = Environment.GetEnvironmentVariable("TENANT_ID");
var projectEndpoint = Environment.GetEnvironmentVariable("AI_FOUNDRY_PROJECT_ENDPOINT");
var modelDeployment = Environment.GetEnvironmentVariable("MODEL_DEPLOYMENT_NAME");

Console.WriteLine($"🔑 Using Tenant ID: {tenantId}");
Console.WriteLine($"🔗 Project Endpoint: {projectEndpoint}");
Console.WriteLine($"🤖 Model Deployment: {modelDeployment}");

// Verify that the environment variables are set
if (string.IsNullOrEmpty(tenantId))
{
    Console.WriteLine("❗ TENANT_ID environment variable is not set.");
    throw new InvalidOperationException("TENANT_ID environment variable is not set.");
}
else if (string.IsNullOrEmpty(projectEndpoint))
{
    Console.WriteLine("❗ AI_FOUNDRY_PROJECT_ENDPOINT environment variable is not set.");
    throw new InvalidOperationException("AI_FOUNDRY_PROJECT_ENDPOINT environment variable is not set.");
}
else if (string.IsNullOrEmpty(modelDeployment))
{
    Console.WriteLine("❗ MODEL_DEPLOYMENT_NAME environment variable is not set.");
    throw new InvalidOperationException("MODEL_DEPLOYMENT_NAME environment variable is not set.");
}
else
{
    Console.WriteLine("✅ All required environment variables are set.");
}


## 🎯 Define Specialist Agent Instructions

Now we'll define the instructions for each of our three specialist agents. Each agent has a specific role in the ticket triage process.

### Priority Assessment Agent

This agent analyzes tickets to determine their urgency level. It categorizes tickets as High, Medium, or Low priority based on their impact on users and business operations.

In [ ]:
// Priority Agent Definition
var priorityAgentName = "priority_agent";
var priorityAgentInstructions = 
    """
    Assess how urgent a ticket is based on its description.

    Respond with one of the following levels:
    - High: User-facing or blocking issues
    - Medium: Time-sensitive but not breaking anything
    - Low: Cosmetic or non-urgent tasks

    Only output the urgency level and a very brief explanation.
    """;

### Team Assignment Agent

This agent determines which team should handle each ticket based on the technical domain and expertise required. It assigns tickets to Frontend, Backend, Infrastructure, or Marketing teams.

In [ ]:
// Team Agent Definition
var teamAgentName = "team_agent";
var teamAgentInstructions =
    """
    Decide which team should own each ticket.

    Choose from the following teams:
    - Frontend
    - Backend
    - Infrastructure
    - Marketing

    Base your answer on the content of the ticket. Respond with the team name and a very brief explanation.
    """;

### Effort Estimation Agent

This agent estimates the amount of work required to resolve each ticket. It categorizes the effort as Small (1 day), Medium (2-3 days), or Large (multi-day/cross-team effort).

In [ ]:
// Effort agent Definition
var effortAgentName = "effort_agent";
var effortAgentInstructions =
    """
    Estimate how much work each ticket will require.

    Use the following scale:
    - Small: Can be completed in a day
    - Medium: 2-3 days of work
    - Large: Multi-day or cross-team effort

    Base your estimate on the complexity implied by the ticket. Respond with the effort level and a brief justification.
    """;

### Orchestrator Agent Instructions

The main orchestrator agent coordinates all three specialist agents. It receives the ticket and uses the specialist agents as tools to provide a comprehensive triage analysis.

In [ ]:
// Instructions for the Orchestrator Agent
var orchestratorInstructions =
    """
    Triage the given ticket. Use the connected tools to determine the ticket's priority, 
    which team it should be assigned to, and how much effort it may take.
    """;

## 🔗 Connect to Azure AI Agent Service

This cell establishes a connection to the Azure AI Agent Service using our project endpoint and credentials. This client will be used to create and manage all our agents.

In [ ]:
// Connect to the agents client using InteractiveBrowserCredential for consistency
var credentialOptions = new InteractiveBrowserCredentialOptions
{
    TenantId = tenantId
};
var credentials = new InteractiveBrowserCredential(credentialOptions);
var client = new PersistentAgentsClient(
    endpoint: projectEndpoint!,
    credential: credentials);

Console.WriteLine("✅ Connected to Persistent Agents Client");

## 🤖 Create Multi-Agent System

This cell creates all four agents in our system:

1. **Three Specialist Agents**: Priority, Team, and Effort assessment agents
2. **Connected Agent Tools**: Wrapper tools that allow the orchestrator to call specialists
3. **Main Orchestrator Agent**: Uses the specialist agents as tools for comprehensive ticket triage

Each specialist agent is created with its specific instructions and then wrapped in a `ConnectedAgentTool` so the orchestrator can use them.

In [ ]:
// Create agents without context manager to keep client active
Console.WriteLine("Creating agents...");

// create the priority agent
var priorityAgent = await client.Administration.CreateAgentAsync(
    model: modelDeployment,
    name: priorityAgentName,
    instructions: priorityAgentInstructions);
Console.WriteLine($"✅ Created Priority Agent with ID: {priorityAgent.Value.Id}");

// Create a connected agent tool for the priority agent
var priorityAgentTool = new ConnectedAgentToolDefinition(
    new ConnectedAgentDetails(
    id: priorityAgent.Value.Id,
    name: priorityAgentName,
    description: "Assesses the priority of a ticket"));
Console.WriteLine($"✅ Created Priority Agent Tool");

// Create the team agent and connected Tool
var teamAgent = await client.Administration.CreateAgentAsync(
    model: modelDeployment,
    name: teamAgentName,
    instructions: teamAgentInstructions);
Console.WriteLine($"✅ Created Team Agent with ID: {teamAgent.Value.Id}");
var teamAgentTool = new ConnectedAgentToolDefinition(
    new ConnectedAgentDetails(
    id: teamAgent.Value.Id,
    name: teamAgentName,
    description: "Determines the appropriate team for a ticket"));
Console.WriteLine($"✅ Created Team Agent Tool");

// Create the effort agent and connected Tool
var effortAgent = await client.Administration.CreateAgentAsync(
    model: modelDeployment,
    name: effortAgentName,
    instructions: effortAgentInstructions);

Console.WriteLine($"✅ Created Effort Agent with ID: {effortAgent.Value.Id}");

var effortAgentTool = new ConnectedAgentToolDefinition(
    new ConnectedAgentDetails(
    id: effortAgent.Value.Id,
    name: effortAgentName,
    description: "Estimates the effort required for a ticket"));

Console.WriteLine($"✅ Created Effort Agent Tool");

// Create the main Agent with the connected tools
var orchestratorAgent = await client.Administration.CreateAgentAsync(
    model: modelDeployment,
    name: "triage-agent",
    instructions: orchestratorInstructions,
    tools: [priorityAgentTool, teamAgentTool, effortAgentTool]);

Console.WriteLine($"✅ Main triage agent created with connected tools: {orchestratorAgent.Value.Id}");
Console.WriteLine("🎯 All agents are ready for use!");

## 💬 Create Conversation Thread

This cell creates a conversation thread that will contain our interaction with the multi-agent system. The thread maintains the context of our conversation and allows for back-and-forth communication.

In [ ]:
// Create the thread for the chat session
Console.WriteLine("Creating agent thread.");
var thread = await client.Threads.CreateThreadAsync();

## 🎯 Execute Multi-Agent Ticket Triage

Now we'll run our multi-agent ticket triage system! This cell will:

1. **Send a ticket**: Submit a sample support ticket to the orchestrator agent
2. **Agent coordination**: The orchestrator will call each specialist agent for their assessment
3. **Display results**: Show the complete conversation flow and final triage analysis

Watch as the orchestrator agent intelligently uses the three specialist agents to provide a comprehensive ticket analysis including priority, team assignment, and effort estimation.

In [ ]:
// Create the ticket prompt
var prompt = "Users can't reset their password from the mobile app.";

// Send the prompt to the orchestrator agent
var message = await client.Messages.CreateMessageAsync(
    thread.Value.Id,
    MessageRole.User,
    prompt);
Console.WriteLine($"💬 Created user message, prompt: {prompt}");

// create and process agent run in thread with tools
Console.WriteLine("🔁 Processing agent run in thread with tools, please wait...");
var run = await client.Runs.CreateRunAsync(thread.Value.Id, orchestratorAgent.Value.Id);
// Poll for run completion
do
{
    await Task.Delay(TimeSpan.FromSeconds(1));
    run = await client.Runs.GetRunAsync(thread.Value.Id, run.Value.Id);
    Console.WriteLine($"🔄 Run status: {run.Value.Status}");
} while (run.Value.Status == RunStatus.Queued || run.Value.Status == RunStatus.InProgress);

Console.WriteLine($"🤖 Run completed with status: {run.Value.Status}");

if (run.Value.Status == RunStatus.Failed)
{
    Console.WriteLine($"‼️ Run failed: {run.Value.LastError?.Message}");
}

// Retrieve and display the messages in the thread
var messages = client.Messages.GetMessagesAsync(
    threadId: thread.Value.Id,
    order: ListSortOrder.Ascending);

Console.WriteLine("\n🗣️ Conversation:");
Console.WriteLine("-------------------");
await foreach (PersistentThreadMessage m in messages)
{
    Console.Write($"🤖 {m.Role.ToString().ToUpper()}: ");
    foreach (MessageContent content in m.ContentItems)
    {
        if (content is MessageTextContent text)
        {
            Console.WriteLine(text.Text);
        }
    }
    Console.WriteLine("-------------------");
}
Console.WriteLine("🎯 Multi-agent ticket triage completed!");

## 🧹 Clean Up Resources

This cell deletes all the agents we created to avoid leaving resources running in Azure. It's important to clean up agents after use to prevent unnecessary costs.

In [ ]:
// Delete the agents when done - Uncomment this block when you're ready to clean up
/*
await client.Administration.DeleteAgentAsync(orchestratorAgent.Value.Id);
await client.Administration.DeleteAgentAsync(priorityAgent.Value.Id);
await client.Administration.DeleteAgentAsync(teamAgent.Value.Id);
await client.Administration.DeleteAgentAsync(effortAgent.Value.Id);
*/